## 瓶颈 IO

### TF数据IO的三种方式
数据直接嵌入graph,由graph传入session中运行


In [2]:
import tensorflow as tf
x = tf.constant([1,2,3],name='x',dtype=tf.float32)
y = tf.constant([2,3,4],name='y',dtype=tf.float32)
z = tf.add(x,y,name='z')
with tf.Session() as sess:
    print (sess.run(z))

[3. 5. 7.]


Pipeline:XXXReader,Queue
Queue机制
1. producer-consumer pattern(生产消费模式)
2. 独立于主线程执行
3. 异步IO:reader.read(queue)tf.train.batch()


- tf.TextLineReader()
- tf.WholeFileReader()
- tf.TFRecordReader()

In [1]:
##异步读取数据
import tensorflow as tf
filenames = tf.train.match_filenames_once('./data/*.csv')

##shuffle是否打乱，num_epochs重复3次,在训练的时候，文件需要过多遍
filename_queue = tf.train.string_input_producer(filenames,shuffle=False,num_epochs=3)

reader = tf.TextLineReader()
_,value = reader.read(filename_queue)

example,label = tf.decode_csv(value,record_defaults=[['null'],['null']])

init_op = tf.local_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)
    coord = tf.train.Coordinator()#线程管理协调器
    threads = tf.train.start_queue_runners(coord=coord) #启动queueRunner
    
    for _ in range(5):
        print(sess.run([example,label]))
        
    coord.request_stop()
    coord.join(threads)



/home/mj/.conda/envs/ten/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>, assertion failed: [string_input_producer requires a non-null input tensor]
	 [[Node: input_producer/Assert/Assert = Assert[T=[DT_STRING], summarize=3, _device="/job:localhost/replica:0/task:0/device:CPU:0"](input_producer/Greater/_11, input_producer/Assert/Assert/data_0)]]


OutOfRangeError: FIFOQueue '_0_input_producer' is closed and has insufficient elements (requested 1, current size 0)
	 [[Node: ReaderReadV2 = ReaderReadV2[_device="/job:localhost/replica:0/task:0/device:CPU:0"](TextLineReaderV2, input_producer)]]

Caused by op 'ReaderReadV2', defined at:
  File "/home/mj/.conda/envs/ten/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/mj/.conda/envs/ten/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/mj/.conda/envs/ten/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/mj/.conda/envs/ten/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/mj/.conda/envs/ten/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/home/mj/.conda/envs/ten/lib/python3.6/site-packages/tornado/platform/as